<h2> Tabular Playground Series-May 2021</h2>

- For this competition, we are being provided with E-commerce data, which is of synthetic in nature and generated by CTGAN, a deep learning technique used to generate data, when the available one is not sufficient to be worked with. 

- In this kernel, I have tried to perform Exploratory Data Analysis followed by Some feature Enginnering, further worked with Modelling. 

- Models are being selected based on there accuracy and Selection of features is done based on SHAP values and those obtained from the Feature Importances. 

<h3><i>If you like or this kernel enhances your knowledges or provides some better insights then do upvote the kernel✔✔</i></h3>

<h3>
<i>The kernel follows a set of steps which are listed as follows:
    
    
1. Data Extraction and Exploration for data insights
    
2. Exploratory Data Analysis
    
3. Feature Selection
    
4. Feature Engineering
    
5. Model Development and Tuning
    
6. Final Model and Result Generation and Submission
    
</i>
    
</h3>

<h1 style="background-color:Violet;" align = 'center'><i>1. Data Extraction and Exploration</i></h1>

In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
##Getting the data in dataframe. 
import seaborn as sns
import matplotlib.pyplot as plt

test = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/test.csv')
train = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/train.csv')
sample = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/sample_submission.csv')

In [ ]:
train.head()

<h4>Id attribute is irrelevant and hence is decided to be dropped from both the sets of training and testing. Also the id in testing set is stored since it is required in result generation</h4>

In [ ]:
##Since the target is categorical, we try encoding them to numerical format.
train.drop('id',axis = 1,inplace = True)
class_dict = {'Class_0':0,'Class_1':1,'Class_2':2,'Class_3':3,'Class_4':4}
train['target_encode'] = train['target'].map(class_dict)

train.drop('target',axis = 1,inplace  =True)

In [ ]:
##Since id is required for result generation, we store it and drop from the test table.
id = test['id']
test.drop('id',axis = 1,inplace = True)

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
test.head()

<h1 style="background-color:DodgerBlue;color:white;" align = 'center'><i>NULL Value Checking📑📍</i></h1>

In [ ]:
sns.heatmap(train.isnull())

<h3> 📌As from the above plot it can be clearly seen that there are no NULL/NaN values in the dataset and hence we are good to procced further to work on the dataset and modelling part</h3>

<h1 style="background-color:SlateBlue;" align = 'center'><i>2. Exploratory Data Analysis🎇✨📊</i></h1>

- Here we try to explore to given training data to get the most of the information from it.
- This is achieved by using various plots using Matplot and seaborn. Most time should be spent on this step since this step is crucial in determining the model accuracy since this is the data being fed to the model for its learning and hence should be wisely done

In [ ]:
X = train.drop('target_encode',axis = 1)
y = train['target_encode']

In [ ]:
for i in X.columns:
    sns.distplot(X[i])
    plt.show()

<h3><b>📌📌 Some points that can be observed from above plots are: </b></h3>


- **From the above plots it can be clearly seen that for almost all the features being selected, we have conclusion that most features have maximum value to be 0 as compared to others i.e the data provided is much of sparse nature**

- Another observation could be done from the plots is that the data is discrete in nature and is of **right skewed**  nature. i.e the data distribution is more aligned to one side of the data instead of being evenly distributed. 

- This might affect our models being trained and hence the data needs to be preprocessed before proceeding for modelling.


<h2>Some more Data Exploration</h2>

- Here we try to visualize the features with negative values these features are being selected from then insights from the above distribution plots

In [ ]:
l_neg = [19,30,31,32,35,38,39,42]
for i in l_neg:
    name = 'feature_'+str(i)
    sns.countplot(X[name])
    plt.show()

In [ ]:
sns.countplot(y)

<h3>From above it is clear that, the target Class_2 is most frequent followed by class_3 and class_4 and then is class_1</h3>

<h1 style = 'background-color:Cornsilk' align = 'center'><i>3. Feature Selection 📋📝 </i></h1>


1. Feature Selection is a necessary concept in Machine Learning since selection of proper and appropriate features leads to improving the models performance and its overall accuracy too
2. Feature Selection is a must done thing when features are more in numbers in any given dataset.Here as can be seen, we have **51 features** and training any model on such large number of features lead to a complex model which is not desirable. 
3. Here we have performed two feature selection techniques, one is by **SHAP values** and other is by using the **feature_importances** attribute in the ExtraTreesClassifier

In [ ]:
 
import xgboost
import shap
# load JS visualization code to notebook
shap.initjs()
# train XGBoost model
X,y = X,y
model = xgboost.train({"learning_rate": 0.01,'max_depth':4}, xgboost.DMatrix(X, label=y), 100)
# explain the model's predictions using SHAP values

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)
shap.summary_plot(shap_values, X, plot_type="bar")


In [ ]:

from sklearn.ensemble import ExtraTreesClassifier
mdl = ExtraTreesClassifier().fit(X,y)
l1 = []
for i,j in zip(X.columns,mdl.feature_importances_):
    
    if(j>0.02):
        print(i,j)
        l1.append(i)
        
l2 = [0,10,12,37,45,25,29]
for i in l2:
    name = 'feature_'+str(i)
    l1.append(name)
    
l1 = sorted(list(set(l1)))

X1 = X[l1]

test1 = test[l1]

In [ ]:
X1.shape,y.shape,test1.shape

<h1 style = 'background-color: PaleGoldenRod' align = 'center'><i>Handling The Right Skewness in the Data</i></h1>

- As mentioned earlier in the kernel, from the graphs it can be clearly seen that we are provided with right skewed data. So inorder to work with the given data, we need to apply some transformations to make the data uniform and useful for the model to be trained with.

- This is done by using Three main transformations as follows:
  - Log Transformation
  - Squareroot Transformation
  - Box-cox Transformation


- All of them require positive data in input and of all, box-cox is efficient and provides similar accuarcy in transformation as in log transformation but is usually preferred to remove skewness from the data.
 
- Here, I have used square root transformation to treat the skewness since we have lot of sparse data and hence other transformations aren't suitable to be applied here. 

In [ ]:

Xtemp = X.copy()
l_neg = [19,30,31,32,35,38,39,42]
lneg = []
for i in l_neg:
    name = 'feature_'+str(i)
    lneg.append(name)
    
cols = [x for x in Xtemp.columns if x not in lneg]
for i in cols:
    Xtemp[i] = np.sqrt(Xtemp[i])

In [ ]:
Xtemp.head()

In [ ]:
testt = test.copy()
for i in cols:
    testt[i] = np.sqrt(testt[i])

In [ ]:
testt.head()

<h3>
📝📝📝Some important things to be considered here is that: 

- We should not always go with Feature Selection directly at times as can be seen here. But we should also look at the attributes nature and accordingly perform it. Here earlier an attempt was performed to select features but after certain trials, it is being recognised that all the features given are relevant and should be considered fo model building and no feature should be eliminated
    
</h3>


<h1 style="background-color:DodgerBlue;" align = 'center'><i>4. Feature Transformation/ Engineering 🛠🔧</i></h1>

- This Step in the Development of Efficient ML Model is performed in order to increase or improve model performance. This includes Scaling the data so that the data adapts to a single common range

- Performing this step is more benefical if performed in case the data is being trained on linear models or the ones where distance is a metric considered for measuring in the model, since there exists a single range of values in the data, the accuracy is significantly affected.

- Some Scalers used here are QuantileTransformation and RobustScaling. 

In [ ]:
from sklearn.preprocessing import QuantileTransformer,RobustScaler
qt = QuantileTransformer()
rs = RobustScaler()
X11 = qt.fit_transform(X)
X2 = rs.fit_transform(X)

In [ ]:
test1 = qt.transform(test)
test2 = rs.transform(test)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y)

X_train1,X_test1,y_train1,y_test1 = train_test_split(X11,y)

In [ ]:
X_train2,X_test2,y_train2,y_test2 = train_test_split(Xtemp,y)

<h1 style="background-color:LightGreen;"align = 'center'><i>5. Model Development and Parameter Tuning</i></h1>

- Considering Different models starting with simple linear ones and then to complex ensemble methods.
- Applying appropriate hyperparameter tuning so that the model does not overfits the data

In [ ]:
#1. Logistic Regression
from sklearn.linear_model import LogisticRegression
mdl = LogisticRegression().fit(X11,y)
mdl.score(X_test1,y_test1)

In [ ]:
#. 2. XGBOOST
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier,StackingClassifier


m2 = XGBClassifier().fit(X11,y)
m2.score(X_test1,y_test1)



In [ ]:
# 3. LightGBM
from lightgbm import LGBMClassifier
model1 = LGBMClassifier(n_estimators = 10000,reg_alpha = 13.156371523046678, reg_lambda = 0.21096899232756502, colsample_bytree = 0.3, subsample = 0.23259307103110016, learning_rate = 0.013455439600227555, max_depth = 39, num_leaves = 305, min_child_samples = 211, min_child_weight = 9.748412613201383e-05,random_state= 13, objective= 'multiclass', metric = 'multi_logloss')
model1.fit(X, y)


In [ ]:
# 3. LightGBM
from lightgbm import LGBMClassifier
model11 = LGBMClassifier(n_estimators = 10000,reg_alpha = 13.156371523046678, reg_lambda = 0.21096899232756502, colsample_bytree = 0.3, subsample = 0.23259307103110016, learning_rate = 0.013455439600227555, max_depth = 39, num_leaves = 305, min_child_samples = 211, min_child_weight = 9.748412613201383e-05,random_state= 13, objective= 'multiclass', metric = 'multi_logloss')
model11.fit(X11, y)


In [ ]:
from sklearn.metrics import log_loss
y_pred2 = model1.predict_proba(X_test)
log_loss(y_test,y_pred2)

In [ ]:
from sklearn.metrics import log_loss
y_pred21 = model11.predict_proba(X_test1)
log_loss(y_test1,y_pred21)

In [ ]:
pred = model1.predict_proba(test)

In [ ]:
pred1 = model11.predict_proba(test1)

In [ ]:
pred

In [ ]:
pred1

<h1 style = 'background-color:Lavender; 'align = 'center'><i>6. Result Generation</i></h1>

In [ ]:
res = pd.DataFrame(pred1,columns = ['Class_1','Class_2','Class_3','Class_4'])
res = pd.concat([id,res],axis = 1)
res.to_csv('result.csv',index = False)
res.head()

<h2 style = 'background-color: BurlyWood;'>
<b>Further Updates to the notebooks will be made soon!</b>
<p> 
    
<i>Any improvement Suggestions are always welcome !😊</i>
    </p>
</h2>
